In [1]:
!pip install pyspark

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import json

In [3]:
spark = SparkSession.builder.appName('Rateio Etapa 2').getOrCreate()

In [4]:
spark

## Funções auxiliares

In [5]:
@F.udf('string')
def data_formato(data):
  if data:
    if '/' in data:
      data_ = data.split('/')
      return data_[-1] + '-' + data_[1] + '-' + data_[0]
  return data

@F.udf('string')
def flag_rateio(rateio):
  if rateio:
    return 'S'
  return 'N'

@F.udf('double')
def valor_pos_rateio(valor, pos_rateio):
  if pos_rateio is None:
    return valor
  return pos_rateio

## Leitura dos dados

In [6]:
json_lancamentos = spark.read.json('/content/data/lancamentos.json')

In [7]:
lancamentos = json_lancamentos.withColumn('array_columns', F.array(*[json_lancamentos.columns]))\
                              .select('array_columns')\
                              .withColumn('columns', F.explode('array_columns'))\
                              .select('columns')\
                              .select("columns.ds_periodicidade","columns.dt_competencia","columns.dt_pagamento","columns.dt_vencimento","columns.id_centro_custo",
"columns.id_centro_resultado","columns.id_forma_pagamento","columns.id_lancamento","columns.valor")

In [8]:
lancamentos.show(5,False)

+----------------+--------------+------------+-------------+---------------+-------------------+------------------+-------------+--------+
|ds_periodicidade|dt_competencia|dt_pagamento|dt_vencimento|id_centro_custo|id_centro_resultado|id_forma_pagamento|id_lancamento|valor   |
+----------------+--------------+------------+-------------+---------------+-------------------+------------------+-------------+--------+
|NULL            |30/11/2024    |NULL        |2024-10-31   |125737         |288                |236252            |47152863     |-125.49 |
|NULL            |31/10/2024    |NULL        |2024-10-31   |32             |902                |236252            |46975400     |-209.13 |
|NULL            |18/09/2024    |2024-09-19  |2024-09-20   |32             |902                |402884            |45095379     |-3540.64|
|NULL            |30/09/2024    |NULL        |2024-09-30   |32             |902                |236252            |45905422     |1.98    |
|NULL            |01/12/202

In [9]:
json_metricas = spark.read.json('/content/data/metricas.json')

In [10]:
metricas = json_metricas.withColumn('array_columns', F.array(*[json_metricas.columns]))\
                              .select('array_columns')\
                              .withColumn('columns', F.explode('array_columns'))\
                              .select('columns')\
                              .select("columns.ds_canal_aquisicao","columns.ds_metrica","columns.ds_segmento","columns.dt_referencia","columns.total")

In [11]:
metricas.show(5,False)

+------------------+----------+-----------+-------------+--------+
|ds_canal_aquisicao|ds_metrica|ds_segmento|dt_referencia|total   |
+------------------+----------+-----------+-------------+--------+
|canalA            |metrica_1 |segmento-S |2024-10-31   |10906.27|
|canalA            |metrica_1 |segmento-M |2024-11-30   |21518.88|
|canalB            |metrica_2 |segmento-R |2022-09-30   |11646.6 |
|canalB            |metrica_3 |segmento-M |2023-06-30   |21305.87|
|canalB            |metrica_3 |segmento-S |2023-06-30   |26511.85|
+------------------+----------+-----------+-------------+--------+
only showing top 5 rows



## Rateio 2 - CENTROS (268 | 288) - Por segmentos
1. Realizar o cast de todos os campos de datas e o campo de valor
1. Filtro dos lançamentos dos meses de OUTUBRO e NOVEMBRO
1. Separar o dataset em 2:
  - Filtro dos centros 268 (centro custo - 125736) e 288 (centro de custo 125737)
  - Restando do dataset fora os centros (268 e 288)
1. Preparar o dataset de métricas, agrupando o valores totais por SEGMENTO (ANO, MES) | TOTAL SEGMENTO (ANO, MES) | TOTAL_METRICA_2 (ANO, MES)
1. Calculo do valor (metrica do segmento / total metrica 2)
1. Join com a tabela de LANÇAMENTOS e calculo com o Valor de Lançamento
1. Criação de colunas de FLAG_RATEIO indicando se houve rateio para aquele dado e coluna VALOR_POS_RATEIO
1. Select e ordenaçao de colunas para uma melhor visualização dos dados
1. Union com os dados que não foram analisados para o rateio
1. Salvamento dos dados .parquet

In [12]:
lanc = lancamentos.withColumn('dt_vencimento',F.col('dt_vencimento').cast('timestamp'))\
                  .withColumn('dt_pagamento',F.col('dt_pagamento').cast('timestamp'))\
                  .withColumn('dt_competencia',data_formato(F.col('dt_competencia')).cast('timestamp'))\
                  .withColumn('valor',F.col('valor').cast('double'))

### Separando os DATASETS

In [13]:
## Parte do dataset para RATEIO
lanc_rateio = lanc.where('month(dt_competencia) = 10 or month(dt_competencia) = 11')\
                   .where('id_centro_resultado = 268 or id_centro_resultado = 288')

In [14]:
## Parte do dataset que NAO sera feito RATEIO
lanc_nao_rateio = lanc.where('(month(dt_competencia) != 10 and month(dt_competencia) != 11) or (id_centro_resultado != 268 and id_centro_resultado != 288)')\
                       .withColumn('flag_rateio', F.lit('N').cast('string'))\
                       .withColumn('valor_pos_rateio', F.col('valor'))\
                       .withColumn('ds_segmento', F.lit(None))\
                       .select(["id_lancamento", "id_centro_custo", "id_centro_resultado", "id_forma_pagamento","ds_periodicidade", "dt_vencimento", "dt_pagamento", "dt_competencia", "valor","ds_segmento", "flag_rateio", "valor_pos_rateio"])

In [15]:
# Count total da tabela
lanc.count()

1312

In [16]:
# Count dos valores para RATEIO
lanc_rateio.count()

142

In [17]:
# Count dos valores para NAO RATEIO
lanc_nao_rateio.count()

1170

### Preparando o dataset de metricas

In [18]:
metr = metricas.where("ds_metrica like 'metrica_2'")\
               .withColumn('dt_referencia', F.col('dt_referencia').cast('timestamp'))\
               .withColumn('total',F.col('total').cast('double'))\
               .withColumn('year', F.month('dt_referencia'))\
               .withColumn('month', F.month('dt_referencia'))\
               .where('month = 10 or month = 11')\
               .where('ds_segmento like "%segmento%"')\
               .select('dt_referencia','ds_segmento','total','year','month')

In [19]:
metr.count()

41

In [20]:
metr.show(12,False)

+-------------------+-----------+-----------------+----+-----+
|dt_referencia      |ds_segmento|total            |year|month|
+-------------------+-----------+-----------------+----+-----+
|2022-10-31 00:00:00|segmento-M |43033.34         |10  |10   |
|2022-11-30 00:00:00|segmento-S |509890.3099999999|11  |11   |
|2022-11-30 00:00:00|segmento-M |43478.65         |11  |11   |
|2022-11-30 00:00:00|segmento-S |484163.4900000003|11  |11   |
|2022-10-31 00:00:00|segmento-M |803912.6300000001|10  |10   |
|2022-10-31 00:00:00|segmento-R |15843.29         |10  |10   |
|2022-10-31 00:00:00|segmento-S |506732.48        |10  |10   |
|2022-10-31 00:00:00|segmento-M |582887.0499999999|10  |10   |
|2022-10-31 00:00:00|segmento-S |468160.9000000006|10  |10   |
|2022-11-30 00:00:00|segmento-M |812105.2599999997|11  |11   |
|2022-11-30 00:00:00|segmento-R |16040.33         |11  |11   |
|2022-11-30 00:00:00|segmento-R |11646.6          |11  |11   |
+-------------------+-----------+-----------------+----

In [21]:
metr_total_dia_segmento = metr.select('dt_referencia','ds_segmento','total')\
                              .groupBy(['dt_referencia','ds_segmento'])\
                              .agg(F.sum("total").alias("total_dia_segmento"))\
                              .withColumn('month', F.month('dt_referencia'))

In [22]:
metr_total_dia_segmento.show(5,False)

+-------------------+-----------+------------------+-----+
|dt_referencia      |ds_segmento|total_dia_segmento|month|
+-------------------+-----------+------------------+-----+
|2023-10-31 00:00:00|segmento-M |1893818.5400000014|10   |
|2024-10-31 00:00:00|segmento-S |1679404.3400000024|10   |
|2024-11-30 00:00:00|segmento-M |2406548.160000001 |11   |
|2024-11-30 00:00:00|segmento-S |1704480.1100000022|11   |
|2022-10-31 00:00:00|segmento-M |1429833.02        |10   |
+-------------------+-----------+------------------+-----+
only showing top 5 rows



In [23]:
total_mes_metrica_2 = metr_total_dia_segmento.withColumn('year', F.year('dt_referencia'))\
                                             .groupBy(['year','month'])\
                                             .agg(F.sum("total_dia_segmento").alias("total_mes_metrica_2"))

In [24]:
total_mes_metrica_2.show(15,False)

+----+-----+-------------------+
|year|month|total_mes_metrica_2|
+----+-----+-------------------+
|2022|10   |2444200.4300000006 |
|2024|10   |5069966.460000005  |
|2022|11   |2464933.98         |
|2023|11   |4382979.360000007  |
|2024|11   |5131834.320000003  |
|2023|10   |4333680.740000005  |
+----+-----+-------------------+



### Calculando -> Valor Rateado para Canal = (Métrica do Canal / Total da Métrica)

In [25]:
rateio_canal_segmento = metr_total_dia_segmento.withColumn('year', F.year('dt_referencia'))\
                                               .join(total_mes_metrica_2, ['year','month'])\
                                               .withColumn('canal_segmento_por_total_metrica',F.col('total_dia_segmento')/F.col('total_mes_metrica_2'))\
                                               .withColumnRenamed('dt_referencia','dt_competencia')\
                                               .select('dt_competencia','ds_segmento','canal_segmento_por_total_metrica')

In [26]:
rateio_canal_segmento.show(5,False)

+-------------------+-----------+--------------------------------+
|dt_competencia     |ds_segmento|canal_segmento_por_total_metrica|
+-------------------+-----------+--------------------------------+
|2023-10-31 00:00:00|segmento-M |0.4370000130651062              |
|2024-10-31 00:00:00|segmento-S |0.33124565088345787             |
|2024-11-30 00:00:00|segmento-M |0.4689450223716497              |
|2024-11-30 00:00:00|segmento-S |0.3321385695086121              |
|2022-10-31 00:00:00|segmento-M |0.584990086103536               |
+-------------------+-----------+--------------------------------+
only showing top 5 rows



In [27]:
rateio = lanc_rateio.join(rateio_canal_segmento, ['dt_competencia'],'left')

In [28]:
# Alguns valores ficaram duplicados, devido a sua condição de ter relação com mais de um segmento.
rateio.count()

158

### Calculo final de RATEIO - relação das métricas * Valor do Lançamento

In [29]:
rateio_final = rateio.withColumn('valor_pos_rateio',F.col('valor')*F.col('canal_segmento_por_total_metrica'))\
                     .withColumn('flag_rateio', flag_rateio(F.col('canal_segmento_por_total_metrica')))\
                     .withColumn('valor_pos_rateio', valor_pos_rateio(F.col('valor'), F.col('valor_pos_rateio')))\
                     .select(["id_lancamento", "id_centro_custo", "id_centro_resultado", "id_forma_pagamento","ds_periodicidade", "dt_vencimento", "dt_pagamento", "dt_competencia", "valor","ds_segmento", "flag_rateio", "valor_pos_rateio"])\
                     .where("flag_rateio like 'S'").orderBy('id_lancamento')

In [30]:
rateio_final.show(10,False)

+-------------+---------------+-------------------+------------------+----------------+-------------------+------------+-------------------+-------+-----------+-----------+-------------------+
|id_lancamento|id_centro_custo|id_centro_resultado|id_forma_pagamento|ds_periodicidade|dt_vencimento      |dt_pagamento|dt_competencia     |valor  |ds_segmento|flag_rateio|valor_pos_rateio   |
+-------------+---------------+-------------------+------------------+----------------+-------------------+------------+-------------------+-------+-----------+-----------+-------------------+
|46975389     |125736         |268                |236252            |NULL            |2024-10-31 00:00:00|NULL        |2024-10-31 00:00:00|-418.28|segmento-R |S          |-84.56295155120213 |
|46975389     |125736         |268                |236252            |NULL            |2024-10-31 00:00:00|NULL        |2024-10-31 00:00:00|-418.28|segmento-M |S          |-195.16361759726507|
|46975389     |125736         |268 

### Union com a outra parte do dataset que não estava dentro das exigências para o RATEAMENTO

In [31]:
save = rateio_final.union(lanc_nao_rateio)

In [32]:
save.count()

1194

In [33]:
save.show(5,False)

+-------------+---------------+-------------------+------------------+----------------+-------------------+------------+-------------------+-------+-----------+-----------+-------------------+
|id_lancamento|id_centro_custo|id_centro_resultado|id_forma_pagamento|ds_periodicidade|dt_vencimento      |dt_pagamento|dt_competencia     |valor  |ds_segmento|flag_rateio|valor_pos_rateio   |
+-------------+---------------+-------------------+------------------+----------------+-------------------+------------+-------------------+-------+-----------+-----------+-------------------+
|46975389     |125736         |268                |236252            |NULL            |2024-10-31 00:00:00|NULL        |2024-10-31 00:00:00|-418.28|segmento-R |S          |-84.56295155120213 |
|46975389     |125736         |268                |236252            |NULL            |2024-10-31 00:00:00|NULL        |2024-10-31 00:00:00|-418.28|segmento-M |S          |-195.16361759726507|
|46975389     |125736         |268 

In [34]:
save.coalesce(1).write.mode('overwrite').parquet('/content/save/etapa2')